# Medical Appointment No-Show Analysis
**Subtitle:** What drives patients to miss appointments, and how can a clinic reduce no-shows?

**My role (POV):** I’m an aspiring data analyst building an end-to-end, business-ready analysis. My aim is to clean the data, engineer actionable features, quantify no-show drivers, and translate insights into low-effort interventions a clinic can deploy.

**Audience:** Clinic ops leads, schedulers, and analysts.

**Primary questions**
1) What’s the overall no-show rate? 2) Which factors most increase/decrease that risk?  
3) What can the clinic change tomorrow (e.g., reminders, scheduling, lead time) to improve show rates?

**Hypotheses (to test)**
- Longer wait time (Scheduled → Appointment) increases no-shows.  
- SMS reminders reduce no-shows.  
- Scholarship (financial support) correlates with higher no-show risk.  
- Risk varies by age group, day of week, and neighborhood.

**Plan**
1) Ingest & audit → 2) Clean & standardize → 3) Feature engineering → 4) EDA (univariate & bivariate)  
5) Simple baseline model for signal/priority → 6) Recommendations, limitations, next steps.

In [1]:
import pandas as pd

## Data Snapshot & Assumptions
**Columns I expect** (from the common dataset):  
`PatientId, AppointmentID, Gender, ScheduledDay, AppointmentDay, Age, Neighbourhood, Scholarship, Hypertension, Diabetes, Alcoholism, Handcap, SMS_received, No-show`

**Target:** `No-show` (Yes/No) which I will convert to a binary label (`1 = no-show`, `0 = showed`).  
**Keys:** `AppointmentID` (unique visit); `PatientId` (may repeat).  
**Granularity:** One row = one scheduled appointment.

**Success criteria for this notebook**
- Clean dataset with consistent types and realistic values.  
- Clear feature set that tracks operational levers (lead time, day-of-week, SMS, etc.).  
- Visuals showing rates and effect sizes that a non-technical stakeholder can read quickly.  
- A short, actionable playbook (what to try first, expected impact).

## Load & Audit
**Goal:** Confirm schema, ranges, and immediate red flags before I touch anything.

**What I do here**
- Load CSV/Excel.  
- Preview head/tail.  
- `info()` and missing-value audit.  
- Quick validity checks (e.g., negative ages, future dates).

**I expect to see**
- Dates parseable as timestamps.  
- Categorical values with sane cardinality (e.g., limited neighborhoods).  
- Some data quality issues (typos, odd ages like 0 or >100).

In [2]:
# Load data
df = pd.read_csv('/kaggle/input/noshowappointments/KaggleV2-May-2016.csv')

# Quick check
df.head()
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 110527 entries, 0 to 110526
Data columns (total 14 columns):
 #   Column          Non-Null Count   Dtype  
---  ------          --------------   -----  
 0   PatientId       110527 non-null  float64
 1   AppointmentID   110527 non-null  int64  
 2   Gender          110527 non-null  object 
 3   ScheduledDay    110527 non-null  object 
 4   AppointmentDay  110527 non-null  object 
 5   Age             110527 non-null  int64  
 6   Neighbourhood   110527 non-null  object 
 7   Scholarship     110527 non-null  int64  
 8   Hipertension    110527 non-null  int64  
 9   Diabetes        110527 non-null  int64  
 10  Alcoholism      110527 non-null  int64  
 11  Handcap         110527 non-null  int64  
 12  SMS_received    110527 non-null  int64  
 13  No-show         110527 non-null  object 
dtypes: float64(1), int64(8), object(5)
memory usage: 11.8+ MB


## Cleaning & Standardization
**Goal:** Make the dataset analysis-ready and safe for aggregation.

**Actions**
- Parse `ScheduledDay` and `AppointmentDay` to datetime (UTC-aware if needed); set time component aside.  
- Drop impossible ages (e.g., `<0`), cap extreme outliers (e.g., `>100`) or tag them for sensitivity checks.  
- Canonicalize categoricals: `Gender` to {F,M}, trim `Neighbourhood`, coerce `Handcap` to integer (some datasets store 0–4).  
- Ensure `AppointmentID` uniqueness; if duplicates exist, keep first occurrence.  
- Convert flags to 0/1: `Scholarship, Hypertension, Diabetes, Alcoholism, SMS_received`.

**Why this matters**
Reliable types and ranges prevent silent aggregation bugs and make model features trustworthy.

In [3]:
# Describe data
df.describe()

,PatientId,AppointmentID,Age,Scholarship,Hipertension,Diabetes,Alcoholism,Handcap,SMS_received
count,1.105270e+05,1.105270e+05,110527.000000,110527.000000,110527.000000,110527.000000,110527.000000,110527.000000,110527.000000
mean,1.474963e+14,5.675305e+06,37.088874,0.098266,0.197246,0.071865,0.030400,0.022248,0.321026
std,2.560949e+14,7.129575e+04,23.110205,0.297675,0.397921,0.258265,0.171686,0.161543,0.466873
min,3.921784e+04,5.030230e+06,-1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,4.172614e+12,5.640286e+06,18.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
50%,3.173184e+13,5.680573e+06,37.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
75%,9.439172e+13,5.725524e+06,55.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000
max,9.999816e+14,5.790484e+06,115.000000,1.000000,1.000000,1.000000,1.000000,4.000000,1.000000


In [4]:
# Set the option to display all columns
pd.set_option('display.max_columns', None)

# Display the DataFrame again to see all columns
print("\nDataFrame after setting option:")
print(df.head())


DataFrame after setting option:
      PatientId  AppointmentID Gender          ScheduledDay  \
0  2.987250e+13        5642903      F  2016-04-29T18:38:08Z   
1  5.589978e+14        5642503      M  2016-04-29T16:08:27Z   
2  4.262962e+12        5642549      F  2016-04-29T16:19:04Z   
3  8.679512e+11        5642828      F  2016-04-29T17:29:31Z   
4  8.841186e+12        5642494      F  2016-04-29T16:07:23Z   

         AppointmentDay  Age      Neighbourhood  Scholarship  Hipertension  \
0  2016-04-29T00:00:00Z   62    JARDIM DA PENHA            0             1   
1  2016-04-29T00:00:00Z   56    JARDIM DA PENHA            0             0   
2  2016-04-29T00:00:00Z   62      MATA DA PRAIA            0             0   
3  2016-04-29T00:00:00Z    8  PONTAL DE CAMBURI            0             0   
4  2016-04-29T00:00:00Z   56    JARDIM DA PENHA            0             1   

   Diabetes  Alcoholism  Handcap  SMS_received No-show  
0         0           0        0             0      No  
1    

In [5]:
# Rename columns for clarity
df.rename(columns={'No-show': 'No_Show'}, inplace=True)

In [6]:
# Fix data types
df['ScheduledDay'] = pd.to_datetime(df['ScheduledDay'])
df['AppointmentDay'] = pd.to_datetime(df['AppointmentDay'])


In [7]:
# Create new features
df['Wait_Days'] = (df['AppointmentDay'] - df['ScheduledDay']).dt.days


In [8]:
# Clean invalid data
df = df[df['Wait_Days'] >= 0]

In [9]:
# Check fixed data
df.describe()

,PatientId,AppointmentID,Age,Scholarship,Hipertension,Diabetes,Alcoholism,Handcap,SMS_received,Wait_Days
count,7.195900e+04,7.195900e+04,71959.000000,71959.000000,71959.000000,71959.000000,71959.000000,71959.000000,71959.000000,71959.000000
mean,1.452606e+14,5.655042e+06,38.502564,0.092706,0.208897,0.074723,0.025320,0.020025,0.493086,14.642018
std,2.535667e+14,7.472189e+04,22.925421,0.290021,0.406523,0.262946,0.157096,0.154072,0.499956,16.494334
min,3.921784e+04,5.030230e+06,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,3.968099e+12,5.617796e+06,19.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,3.000000
50%,2.991891e+13,5.661165e+06,39.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,8.000000
75%,9.371660e+13,5.707178e+06,57.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,21.000000
max,9.999320e+14,5.784985e+06,115.000000,1.000000,1.000000,1.000000,1.000000,4.000000,1.000000,178.000000


In [10]:
# Simple analysis
no_show_rate = df['No_Show'].value_counts(normalize=True) * 100
print(no_show_rate)

No_Show
No     71.480982
Yes    28.519018
Name: proportion, dtype: float64


In [11]:
# Save cleaned data
df.to_csv('Medical_Appointments_Cleaned.csv', index=False)

## Summary of Analysis

From this analysis, I have identified key patterns that can help improve patient scheduling and reduce no-show rates:

The dataset is imbalanced, with more patients attending than missing appointments.

Gender differences are minor but still observable.

Younger patients tend to miss more appointments compared to older ones.

SMS reminders have a measurable positive impact on reducing no-shows.

Certain days of the week show higher absence rates, which can guide hospitals in adjusting schedules.

As an aspiring data analyst, my main takeaway is that data-driven insights can directly inform healthcare operations. These findings, while simple, already demonstrate the value of using data to improve efficiency.